In [1]:
import os
import io
import time
import os.path as osp

import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from torchvision import transforms
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize

from PIL import Image, ImageFilter
from PIL import Image

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
parquet_path = "data/pokemon.parquet"
df = pd.read_parquet(parquet_path)
df

,image,text
0,{'bytes': b'\xff\xd8\xff\xe1#\rExif\x00\x00MM\...,a drawing of a green pokemon with red eyes
1,{'bytes': b'\xff\xd8\xff\xe1#\x98Exif\x00\x00M...,a green and yellow toy with a red nose
2,{'bytes': b'\xff\xd8\xff\xe1\x18\x82Exif\x00\x...,a red and white ball with an angry look on its...
3,{'bytes': b'\xff\xd8\xff\xe1\x1a\x06Exif\x00\x...,a cartoon ball with a smile on it's face
4,{'bytes': b'\xff\xd8\xff\xe1\x1d\xe3Exif\x00\x...,a bunch of balls with faces drawn on them
...,...,...
828,{'bytes': b'\xff\xd8\xff\xe1\x1e\xb1Exif\x00\x...,a cartoon picture of a stone dragon
829,{'bytes': b'\xff\xd8\xff\xe1\x1a/Exif\x00\x00M...,a drawing of a yellow and gray pokemon character
830,{'bytes': b'\xff\xd8\xff\xe1!\xcdExif\x00\x00M...,a drawing of a cat with a wheel in his hand
831,{'bytes': b'\xff\xd8\xff\xe1\x1cLExif\x00\x00M...,a drawing of a cartoon character with two eyes


In [4]:
import pickle
from tqdm import tqdm


# Define CLIP transform
transform = Compose([
    Resize((256, 256)),
    CenterCrop(256),
#     ToTensor(),
#     Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
#               std=[0.26862954, 0.26130258, 0.27577711])
])

# List to store preprocessed data
preprocessed_data = []

for index, row in tqdm(df.iterrows()):
    # Load and preprocess image
    image_bytes = row['image']['bytes']
    
    # Convert bytes to image
    image = Image.open(io.BytesIO(image_bytes))
    
    # Apply transformation
    image = transform(image)

    # Preprocess text
    text = row['text'] 

    # Add to the list
    preprocessed_data.append([image,text])

833it [00:08, 99.50it/s] 


In [5]:
# Save preprocessed_data to a file using pickle
with open('data/preprocessed_data.pkl', 'wb') as file:
    pickle.dump(preprocessed_data, file)